# Workgroup 4

Authors: Valerie Dube, Erzo Garay, Juan Marcos Guerrero y Matias Villalba

## Bootstraping

## Causal Forest

In [1]:
set.seed(1)
rm(list = ls())

In [11]:
# Installing packages
install.packages("grf")
install.packages("Hmisc")
install.packages("evaluate")

also installing the dependencies ‘zoo’, ‘DiceKriging’, ‘lmtest’, ‘Rcpp’, ‘sandwich’, ‘RcppEigen’


Warning message in install.packages("grf"):
“installation of package ‘zoo’ had non-zero exit status”
Warning message in install.packages("grf"):
“installation of package ‘DiceKriging’ had non-zero exit status”
Warning message in install.packages("grf"):
“installation of package ‘Rcpp’ had non-zero exit status”
Warning message in install.packages("grf"):
“installation of package ‘lmtest’ had non-zero exit status”
Warning message in install.packages("grf"):
“installation of package ‘sandwich’ had non-zero exit status”
Warning message in install.packages("grf"):
“installation of package ‘RcppEigen’ had non-zero exit status”
Warning message in install.packages("grf"):
“installation of package ‘grf’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘fastmap’, ‘fs’, ‘rappdirs’, ‘farver’, ‘munsell’, ‘string

In [14]:
# Libraries
library(grf)
library(sandwich)
library(lmtest)
library(Hmisc)
library(ggplot2)

ERROR: Error in library(grf): there is no package called ‘grf’


### 1. Preprocessing

In [15]:
# Import synthetic data from data folder
data.all = read.csv("../../data/synthetic_data.csv")

data.all$schoolid = factor(data.all$schoolid)

In [19]:
DF = data.all[,-1]

In [20]:
school.id = as.numeric(data.all$schoolid)

school.mat = model.matrix(~ schoolid + 0, data = data.all)
school.size = colSums(school.mat)

In [8]:
# It appears that school ID does not affect pscore. So ignore it
# in modeling, and just treat it as source of per-cluster error.
w.lm = glm(Z ~ ., data = data.all[,-3], family = binomial)
summary(w.lm)

W = DF$Z
Y = DF$Y
X.raw = DF[,-(1:2)]

C1.exp = model.matrix(~ factor(X.raw$C1) + 0)
XC.exp = model.matrix(~ factor(X.raw$XC) + 0)

X = cbind(X.raw[,-which(names(X.raw) %in% c("C1", "XC"))], C1.exp, XC.exp)


Call:
glm(formula = Z ~ ., family = binomial, data = data.all[, -3])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2079  -0.9088  -0.8297   1.4176   1.9556  

Coefficients: (6 not defined because of singularities)
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.9524636  0.2845173  -3.348 0.000815 ***
schoolid2    0.0697302  0.2766287   0.252 0.800986    
schoolid3    0.0382080  0.2911323   0.131 0.895586    
schoolid4    0.1761334  0.2784711   0.633 0.527059    
schoolid5   -0.0033389  0.2950180  -0.011 0.990970    
schoolid6    0.0583548  0.3067481   0.190 0.849124    
schoolid7   -0.1313759  0.3188190  -0.412 0.680288    
schoolid8    0.1233661  0.3023736   0.408 0.683279    
schoolid9   -0.1955428  0.3073344  -0.636 0.524611    
schoolid10  -0.1892794  0.2968750  -0.638 0.523752    
schoolid11  -0.2224060  0.5461005  -0.407 0.683816    
schoolid12  -0.3312420  0.5414374  -0.612 0.540682    
schoolid13  -0.0408540  0.3989507  -0.102 0.9

### 2. Causal Forest estimation and results

#### 2.1. Causal Forest

In [9]:
#
# Grow a forest. Add extra trees for the causal forest.
#

Y.forest = regression_forest(X, Y, clusters = school.id, equalize.cluster.weights = TRUE)
Y.hat = predict(Y.forest)$predictions
W.forest = regression_forest(X, W, clusters = school.id, equalize.cluster.weights = TRUE)
W.hat = predict(W.forest)$predictions

cf.raw = causal_forest(X, Y, W,
                       Y.hat = Y.hat, W.hat = W.hat,
                       clusters = school.id,
                       equalize.cluster.weights = TRUE)
varimp = variable_importance(cf.raw)
selected.idx = which(varimp > mean(varimp))

cf = causal_forest(X[,selected.idx], Y, W,
                   Y.hat = Y.hat, W.hat = W.hat,
                   clusters = school.id,
                   equalize.cluster.weights = TRUE,
                   tune.parameters = "all")

tau.hat = predict(cf)$predictions

ERROR: Error in regression_forest(X, Y, clusters = school.id, equalize.cluster.weights = TRUE): could not find function "regression_forest"
